Regular Expressions
===================

...or, How to find all of what you're looking for.

Idea behind REs is to do one search to find, not a word or sequence of words, but a *pattern*. We've all had times when we want to change a word in a document and we have to do several searches, e.g. "give", "gave", "given", before we're sure we have them all. Today we'll learn how to do it more quickly, efficiently, and sometimes correctly!

In [ ]:
import re   # 're' stands for 'regular expressions'

The first thing we need to do is to read in some text to work with. Here is a plain-text file with the contents of _Alice in Wonderland_.

In [ ]:
fh = open( "../lessondata/alice.txt", "r" )
alicetext = fh.read()
fh.close()

Did you get an error when you tried this? You might have to step back and think about about file encodings. Your error might be telling you that the text file is not all in plain old English characters, so you have to know how it was encoded. In this case, the file uses the UTF-8 Unicode encoding.

What do you suppose we need to do to fix it?

In [ ]:
fh = open( "../lessondata/alice.txt", "r", encoding="utf-8" )
alicetext = fh.read()
fh.close()
print(alicetext)

That's better! Now that we have the text, we can talk about regular expressions. The simplest regular expression is just a word, or a set of words, or a part of a word, that you want to search for. Let's search in the text for the word 'give'.

In [ ]:
result = re.search( "give", alicetext )
print(result)
print(result.start())
print(alicetext[result.start():result.end()])

The `.search()` method returns an answer the first time it finds what we're looking for. Here we found a place in the text where the word 'give' occurs, and we printed it out.

Regular expressions can get pretty complex! For example:

In [ ]:
string = "My email address is tara.andrews@dh.unibe.ch, " +\
    "and another email address I have is tla@mit.edu."

email_re = "[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}"
print(re.findall(email_re , string ))

That is an example of a regular expression to match most valid email addresses. Pretty horrible to look at but gets the job done.

You can see in the second example that we usually want all the matches, not just the first one. So we can use `.findall()` instead of `.search()`. 

In [ ]:
result = re.findall( "give", alicetext )
print(result)

If we use a slightly different function, `.finditer()`, we can go through these results to generate a concordance. For example:

In [ ]:
result = re.finditer( "give", alicetext )
for match in result:
    begin = match.start() - 30
    end = match.end() + 30
    print("-----")
    print(alicetext[begin:end])

If we remove the line breaks, we'll have written our very own concordance tool!

In [ ]:
result = re.finditer( "give", alicetext )
for match in result:
    begin = match.start() - 30
    end = match.end() + 30
    context = alicetext[begin:end].replace('\n', ' ')
    print("-----")
    print(context)

One of those results has actually returned 'given' rather than 'give'. And what about 'gave'? This is where regular expressions get more interesting.

One option is to just list out all the variants you might be looking for, separated by this | character...

In [ ]:
print(re.findall( "(given|gave|gives|give|giving)", alicetext))

But that only got us the lowercase versions, and maybe we want all versions of 'give' no matter the case. That means we have to provide a _flag_.

In [ ]:
print(re.findall( "(given|gave|gives|give)", alicetext, re.I ))

And if we want to see some of the power of what regular expressions can really do, we can say "Well all versions if this word start with a 'g' and have a vowel that's either 'i' or 'a', then a 've', and then maybe an 'n' or an 's'. We can write the regular expression to look for a *pattern* like this.

In [ ]:
print(re.findall( "(g[ia]v(e[ns]?|ing))", alicetext, re.I ))

So what on earth is that? Here is where we talk about flags and metacharacters. In this expression, the flag is `re.I` and the metacharacters were `[]` and `?`.

The [] means "here I expect a single character that might be anything I've listed between the brackets."
The ? means "The character I just told you about may or may not be there; match either way.
The re.I at the end means "We don't care whether it's capital or lowercase."

So we've said we want a 'g', followed by either an 'i' or an 'a', followed by an 've', and possibly (but not necessarily) ending in an 'n' or an 's', and that we don't care what case the word is in.

Flags
-----
Regular expressions usually take some set of *flags* that alter how the expression is treated. The most useful one to know about is the one we used:

    re.I   (Case-insensitive: don't pay attention to upper- or lowercase)
    

Characters, metacharacters, and patterns
----------------------------------------

A regular expression is a *pattern* specified using *characters* and *metacharacters*. A character is, well, any old thing that can appear in a text file. A metacharacter is a character that doesn't get treated as itself, but rather signals to the regular expression engine that you want to express something more complicated. Typical metacharacters are:

    .       (Match any character)
    [,+;]   (Match the character if it is any of the things inside the [])
    (abc)   (Make abc a group: apply any of the following to the whole thing.)
    +       (Match the previous character or group one or more times)
    {3}     (Match the previous character or group exactly three times)
    {1,4}   (Match the previous character or group between 1 and 4 times)
    *       (Match the previous character or group zero or more times)
    ?       (If the previous character or group isn't there, treat the pattern as a match anyway)
    \       (The thing that follows is a metacharacter (if normally not) or a character (if normally meta))
    
So this means that:

* `(abc)+` will match `abc` or `abcabc` but not `abac`.
* `[abc]+` will match `a` or `b` or `abac` or really any combination of a, b, and c.
* If you want to match anything at all, you match `.*`. 
* If you want to match anything except the empty string, you match `.+`.
* If you want to match a period, you match `\. `; for a plus sign, `\+ `.

Let's try it out:

In [ ]:
print(re.findall( "[A-Za-z]+!", alicetext ))

Here we have looked for all words that end in an exclamation point...

* any character from A-Z, or from a-z ... `[A-Za-z]`
* matched multiple times ... `+`
* followed by an exclamation point ... `!`

There is an easier way to specify this, relying on some more of these metacharacters. The three most important are:

    \w   (match a "word" character, which is usually A-Z, a-z, 0-9, and _)
    \d   (match a "digit" character, which is generally 0-9)
    \s   (match any sort of "space" character, including space, tab, carriage return, etc.)

So we can say instead:

In [ ]:
print(re.findall( "\w+!", alicetext ))

We can also see if there are any numbers in the text:

In [ ]:
print(re.findall( "\d+", alicetext ))

We can also find things depending on whether they are at the beginning or the end of the line. Here are two more metacharacters:

    ^   The beginning
    $   The end
    
So we can make a listing of chapters by searching through the file for where 'chapter' appears at the beginning of the line.

In [ ]:
for heading in re.finditer( "^CHAPTER .*$", alicetext):
    print(heading.group(0))

Hmmmm, that didn't work. Why not? Well, what word(s) does Python think is at the beginning?

In [ ]:
print(re.findall("^\w+", alicetext))

Aha. That's the single word (well, part of a word) that is at the beginning of the whole "alicetext" string! But we really want `^` and `$` to match the beginning and end of every line. So we need another flag:

    re.M   Multi-line mode: ^ and $ apply to every line, not just the string itself.
    
When we use this flag, we change what `^` and `$` mean, and get what we expect.

In [ ]:
for heading in re.finditer( "^CHAPTER .*$", alicetext, re.M):
    print(heading.group(0))

What is this 'group' thing? It's another feature of regular expressions: every time you put something in parentheses in a regular expression, you can get at it separately from the rest of the thing you matched. The 0th group is always the whole match, and then the groups are numbered in the order that their parentheses start.

This is very useful if, for instance, you want to print out the chapter numbers and titles by themselves. We just put the part we want to keep in parentheses, like so.

In [ ]:
chapterlist = {}

for heading in re.finditer( "^CHAPTER ([IVX]+)\.\s+(.*)$", alicetext, re.M):
    chapterlist[heading.group(1)] = heading.group(2)
    
for k in sorted(chapterlist):
    print("%-4s : %s" % (k, chapterlist[k]))

The regular expressions module also has another very useful feature, which is that you can not only find things, you can replace them. This is the `sub` function, meaning "substitute".

Let's say that Alice has reached her teenage years and is exploring her identity, and wants to be known for a while by her middle name, Pleasance. We can fix the text by substituting the new name for the old.

In [ ]:
ptext = re.sub( "Alice", "Pleasance", alicetext, flags=re.I)
print(ptext[:1000])

The third useful thing you can do with regular expressions, that will be important for the homework, is to use a pattern to split up some text. Let's say that you have a shopping list that looks like this:

3 bananas, 3 apples, 500g steak, 1 bottle of beer

and you want to make a proper list out of it. You'll need to do some matching and some splitting up!

First we can see that the list is separated by commas. We can split it like so:

In [ ]:
shopping_list = "3 bananas, 3 apples, 500g steak,  6 bottles of beer"
things = re.split(',\s+', shopping_list)
for item in things:
    print(item)

This looks an awful lot like the string splitting function that we already knew about. But what if we got this list from OCR, and it looks rather more untidy? Here is where regular expressions help a lot more.

In this case we want to split up the list, but in doing that we want to throw away any commas or periods, as well as empty space before and after them.

In [ ]:
shopping_list = "3 bananas. 3 apples ... 500g steak,6 bottles of beer  "
things = re.split('\s*[.,]+\s*', shopping_list)
for item in things:
    print(item + '!')

We still have the problem of that empty space after the beer. So when we are cleaning up text, the first thing we almost always do is to get rid of any space at the beginning and at the end. This is something you will see (and do) a lot when using regular expressions on real text.

In [ ]:
shopping_list = "3 bananas. 3 apples ,, 500g steak, 6 bottles of beer  "
## These lines, or lines like them, are used a lot!! 
## Don't forget the re.M flag if you want to do this for all the lines in a file.
shopping_list = re.sub("^\s+", '', shopping_list)
shopping_list = re.sub("\s+$", '', shopping_list)
## Now we can get on with the rest of the program.
things = re.split('\s*[.,]+\s*', shopping_list)
print(things)

Okay, we have our list, but we should be able to split it up into "quantity" and "item". We can observe a few rules:

- Quantities are usually numbers, but might be amounts like '500g' or '6 bottles'.
- If the quantity is more than one word, there will generally be an 'of' in there somewhere.
- If we can't split on the 'of', then we just split the words.

Let's do this!

In [ ]:
shopping = {}

for thing in things:
    if re.search('of', thing):
        parsed = re.split('\s+of\s+', thing)
    else:
        parsed = thing.split()
    shopping[parsed[1]] = parsed[0]
    
print(shopping)

In [ ]:
from IPython.display import Image

Image(url="http://imgs.xkcd.com/comics/regular_expressions.png")